# Import Library

The YOLOP network and CNN + LSTM are two different NN trained in different time.
For the testing or training phase is necessary download the code of YOLOP showed in the readme and used it for preprocessing the data sample for this network.

In [ ]:
import sys
!export TF_FORCE_GPU_ALLOW_GROWTH='true'

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow as tf
from tensorflow.python.client import device_lib

sess = InteractiveSession()
sess.close()
print(tf.test.gpu_device_name())
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

 #!{sys.executable} -m pip install opencv-python
import sklearn
import os
from os import listdir
from os.path import isfile, join, isdir
import gc
import sklearn.model_selection

from pathlib import Path

import cv2
import numpy as np

import sklearn
import os
from os import listdir
from os.path import isfile, join, isdir
import sklearn.model_selection
from pathlib import Path
import imutils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL
from cv2 import cv2

import cv2
import numpy as np

from tensorflow.keras.layers import Input, Flatten, LSTM, Reshape, TimeDistributed
from tensorflow.keras.layers import GlobalMaxPooling1D, BatchNormalization, TimeDistributed
from tensorflow.keras.layers import GRU, Dense, Dropout, LSTM, Input, concatenate, Conv2D, MaxPool2D
from tensorflow.keras.layers import GlobalMaxPool2D, ZeroPadding1D
from tensorflow import keras
from tensorflow.keras import Model


# Dataset

In [ ]:
W = 50
H = 50
channels = 2

nbout=4
NF = 24


HEIGHT, WIDTH, CHANNELS = (150,150, 4)#(int(736/4), int(1280/4))
NBFRAME = NF
idx = 30


class dataset():
    def __init__(self, directory):
        
        folder = []
        ##=============================== GET PATH AND LABEL OF FILES ===============================
        for i in directory:
            folder.append(Path(i))

        # obtain all the filenames of files inside all the class folders 
        # going through each class folder one at a time
        self.fnames, labels = [], []
        for i in folder:
            for label in sorted(os.listdir(i)):
                self.fnames.append(os.path.join(i, label))#, fname))
                for fname in os.listdir(os.path.join(i, label)):
                    labels.append(label)
                
        # prepare a mapping between the label names (strings) and indices (ints)
        self.label2index = {label:index for index, label in enumerate(sorted(set(labels)))} 
        
        # convert the list of label names into an array of label indices
        self.label_array1 = np.array([self.label2index[label] for label in labels], dtype=int)        
        self.label_array = []
        for i in np.arange(len(self.label_array1)):
            self.label_array.append(np.zeros(4))
            
            self.label_array[i][self.label_array1[i]] = 1
        #create_dataset()
    
        self.data_array1 = []
        self.data_array2 = []
        self.data_array3 = []
        
        old = 0
    
        for i in np.arange(len(self.fnames)):
            for folname in os.listdir(self.fnames[i]):
                fname = os.path.join(self.fnames[i], folname)
                files = [f for f in os.listdir(fname) if os.path.isfile(os.path.join(fname, f))]
    
                clips1 = []
                clips2 = []
                clips3 = []
                
                for f in files:
#==========================================load del file f====================================================
                    try:
                        clip = np.load(os.path.join(fname,f), allow_pickle=True)
                        read_wrong = False
                    except:
                        read_wrong = True
                        print("error")

                    # extract 3 kind of infomation:
                    #     x1 = car bounding box
                    #     x2 = road drivable segmentation
                    #     x3 = lane linee segmentation
                    if(read_wrong == False):
                        x1_ = clip[0][0]

                        img = clip[0][1]
                        
                    clips1.append(np.array((x1_)))#.ravel(), clip[0][1].ravel(), clip[0][2].ravel(), clip[0][3].ravel())))
                    clips2.append(np.array((img)))
                                    
                self.data_array1.append(clips1)
                self.data_array2.append(clips2)
                                                
                old=int(folname)
                clips1 = []
                clips2 = []
                
        print1 = self.data_array1.copy()
        print2 = self.data_array2.copy()
        
        self.data_array1 = np.zeros((np.shape(self.data_array1)[0], NF, idx, 7, 1))
        self.data_array2 = np.zeros((np.shape(self.data_array2)[0], NF, W, H, channels))
        
        for i in np.arange(np.shape(self.data_array1)[0]):
            try:
                self.data_array1[i] = print1[i]
            except:
                print("error")
            try:
                self.data_array2[i] = print2[i]
            except:
                print("error")
            
        self.data_array1 = np.array(self.data_array1)
        self.data_array2 = np.array(self.data_array2)
        
    def getitem(self):
        
        x1_TR, x1_TS, x2_TR, x2_TS, y_TR, y_TS = sklearn.model_selection.train_test_split(self.data_array1,self.data_array2, self.label_array, test_size=0.25)
        
        return x1_TR, x1_TS, x2_TR, x2_TS, y_TR, y_TS
    
    def len(self):
        return len(self.data_array1)
    
    
        


# Implement Network

In [ ]:
def LSTM_NET():

    momentum = .9

    in1 = Input((idx, 7, 1))

    in2 = Input((W, H, 2))

    flt1 = Flatten()(in1)

    x2 = Conv2D(64, (3,3), input_shape=((W,H,2)),padding='same', activation='relu')(in2)
    x2 = BatchNormalization(momentum=momentum)(x2)
    x2 = MaxPool2D()(x2)
    x2 = Conv2D(128, (3,3), padding='same', activation='relu')(x2)
    x2 = BatchNormalization(momentum=momentum)(x2)
    x2 = MaxPool2D()(x2)
    x2 = Conv2D(256, (3,3), padding='same', activation='relu')(x2)
    x2 = BatchNormalization(momentum=momentum)(x2)
    x2 = GlobalMaxPool2D()(x2) #vettore in uscita pari a 128
    x2 = Dense(128, activation='relu')(x2)
    x2 = Dropout(.2)(x2)
    x2 = Dense(64, activation='relu')(x2)
    x2 = Dropout(.2)(x2)
    x2 = Dense(32, activation='relu')(x2)
    x2 = Dropout(.2)(x2)
    x2 = Dense(25, activation='relu')(x2)
    
    preModel1 = Model(inputs=in1, outputs = flt1)
    preModel2 = Model(inputs=in2, outputs = x2)

    in1 = tf.keras.Input(shape=(NF, idx, 7, 1))
    in2 = tf.keras.Input(shape=(NF, W, H, 2))
    t1 = tf.keras.layers.TimeDistributed(preModel1)(in1)
    t2 = tf.keras.layers.TimeDistributed(preModel2)(in2)

    x = concatenate([t1, t2])

    x = LSTM(int(235*2/3), return_sequences=True)(x)
    x = LSTM(int(235*4/9))(x)

    #x = GlobalMaxPooling1D()(x)

    x = Dense(1024, activation='relu')(x)
    x = Dropout(.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(nbout, activation='softmax')(x)

    model = Model(inputs=[in1, in2], outputs = x)

    model.summary()
    return model

In [ ]:
NF = 80#number of frames used
NBFRAME = NF
        
path = ["Dataset1", "Dataset2"]

#Creating dataset
x = dataset(path)
x1_TR, x1_TS, x2_TR, x2_TS, y_TR, y_TS = x.getitem()

x1_TR = np.asarray(x1_TR)
x2_TR = np.asarray(x2_TR)

x1_TS = np.asarray(x1_TS)
x2_TS = np.asarray(x2_TS)
        
y_TR = np.asarray(y_TR)
y_TS = np.asarray(y_TS)

#Creating model
model = LSTM_NET()

#Launch training
optimizer = keras.optimizers.Adam(0.001)
        
model.compile(
    optimizer,
    'categorical_crossentropy',
    #loss = 'mse',
    metrics=['acc']
)
        
EPOCHS=300

try:
    os.mkdir(str(NF) + ' frames/'+'chkp_'+str(i)+'_'+str(NF))
    os.mkdir(str(NF) + ' frames/'+'chkp_'+str(i)+'_'+str(NF)+'/logs')
except:
    print("folder chkp already exist")
        
callbacks = [keras.callbacks.ModelCheckpoint(str(NF) + ' frames/'+'chkp_'+str(i)+'_'+str(NF)+'/chkp'+str(NF)+'weights.{epoch:02d}-{val_loss:.2f}.hdf5',verbose=1),keras.callbacks.TensorBoard(log_dir=str(NF) + ' frames/'+'chkp_'+str(i)+'_'+str(NF)+'/'+'/logs')]

model.fit(
    [x1_TR, x2_TR],
    y_TR,
    batch_size=12,
    validation_data=((x1_TS, x2_TS), y_TS),
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)